In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *

In [14]:
macd_indicator = Indicator.objects.get(name="MACD")
sorted_stock = SortedStocksList.objects.get(symbol__symbol="ioc", created_at__date=datetime.now().date())
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
# df = df.loc[df["date"] >= last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df.loc[(df.stoch_diff <= -22.70)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df.loc[(df.stoch_diff >= 0.070)].iloc[0]
except:
    crossover_signal = None
# df.head(50)
df.loc[(df.stoch_diff <= -22.70)]

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
75,M5,117.8,117.95,116.05,116.40,3639423,None,None,2020-01-20 09:15:00,5.109489,43.369830,-38.260341,-1.954908,SELL
97,M5,118.8,118.80,118.35,118.40,438983,None,None,2020-01-20 11:05:00,26.470588,51.786492,-25.315904,-6.701269,SELL
98,M5,118.4,118.40,118.00,118.05,638430,None,None,2020-01-20 11:10:00,3.030303,30.574371,-27.544068,-0.834669,SELL


In [ ]:
sorted_stock = SortedStocksList.objects.get(symbol__symbol="ioc", created_at__date=datetime.now().date())
get_macd_crossover(sot)

In [15]:
df

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
0,M5,124.25,124.25,123.55,123.60,185443,None,None,2020-01-17 09:15:00,7.142857,7.142857,0.000000,0.000000,BUY_CROSSOVER
1,M5,123.60,123.75,123.35,123.65,196869,None,None,2020-01-17 09:20:00,33.333333,20.238095,13.095238,4.365079,BUY
2,M5,123.65,123.65,123.50,123.60,122859,None,None,2020-01-17 09:25:00,27.777778,22.751323,5.026455,1.396238,BUY
3,M5,123.60,123.75,123.50,123.55,92840,None,None,2020-01-17 09:30:00,22.222222,27.777778,-5.555556,-1.234568,SELL_CROSSOVER
4,M5,123.55,123.70,123.55,123.65,83088,None,None,2020-01-17 09:35:00,33.333333,27.777778,5.555556,1.851852,BUY_CROSSOVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,M5,117.50,117.55,117.20,117.40,700717,None,None,2020-01-20 15:00:00,23.076923,24.358974,-1.282051,-0.295858,SELL_CROSSOVER
145,M5,117.40,117.55,117.30,117.45,631000,None,None,2020-01-20 15:05:00,26.923077,25.641026,1.282051,0.345168,BUY_CROSSOVER
146,M5,117.40,117.50,117.25,117.45,644622,None,None,2020-01-20 15:10:00,26.923077,25.641026,1.282051,0.345168,BUY
147,M5,117.45,117.70,117.40,117.70,1197942,None,None,2020-01-20 15:15:00,46.153846,33.333333,12.820513,5.917160,BUY
